# Testing and verifying accuracy of Infiflux


In [ ]:
import pandas as pd
import yfinance as yf
import pytz
import numpy as np
from IPython.display import clear_output
import time
from datetime import datetime, timedelta

In [ ]:
# download it form the platform.
# date that needs to be checked
chck = '2024-05-08'      # Please change the date required to be checked.

# time interval to collect data that includes adjacent dates.
start = str(datetime.strptime(chck, '%Y-%m-%d')- timedelta(days = 4)).split(' ')[0]
end = str(datetime.strptime(chck, '%Y-%m-%d') + timedelta(days = 2)).split(' ')[0]

# file available in github, one can check the accuracy for respective dates,
# change the value of the chck variable
df = pd.read_excel('data.xlsx', index_col=[0])                     # download the file from Infiflux.com using your account
df = df[df.Date.str.contains(chck)].copy()
yfinance = [ [i.replace('(','').replace(')','').replace('Â\xa0',''),i] for i in df.Yfinance]

# downloading and collecting the data.
df_li = {}
for i in yfinance:
  df_li[i[1]] =  yf.download(i[0],start = start, end = end,interval = '1m')
  # time.sleep(0.9)

clear_output(wait=False)
print('done')

done


In [ ]:
# assuming every prediction is incorrecting from the start
# and then updating only those which are correct later.
df['DayBreak_check'] = 0
df['MidDay_check'] = 0

df_li_tmp = df_li.copy()
for i in df_li.keys():
  fd = df_li[i].copy()
  fd['date'] = [str(j) for j in fd.index]
  dt = pd.DataFrame([ j.split(' ')[0] for j in fd['date']])[0].unique()
  fd = pd.concat([
    fd[fd.date.str.contains(dt[list(dt).index(chck) ])],
    fd[fd.date.str.contains(dt[list(dt).index(chck) + 1])]
    ])
  # accuracy check
  # first quarter of the next day
  Q1 = fd[fd.date.str.contains(dt[list(dt).index(chck) + 1])].copy()
  Q1 = Q1[:int(len(Q1)/4)]
  # second half of the next day
  S2 = fd[fd.date.str.contains(dt[list(dt).index(chck) + 1])].copy()
  S2 = S2[int(len(S2)/2):]
  # previous day close
  Q1_prevdayclose = fd[fd.date.str.contains(dt[list(dt).index(chck)])].iloc[-1].Close
  # next day first quarter close
  Q1_close = Q1.iloc[-1].Close

  # each instrument row to check the correctness of the forecast
  check = df[df.Yfinance == i]

  for index,row in Q1.iterrows():
    if check.DayBreak.iloc[0] == 'Bullish':
      # Daybreak condition if bullish, previous day close should be less than next predicted day close any where
      # in the first quarter of the day
      if row.Close > Q1_prevdayclose:
        df.loc[df.Yfinance == i, 'DayBreak_check'] = 1
        #correct condition.
        break
    elif check.DayBreak.iloc[0] == 'Bearish':
      # Daybreak condition if Bearish, previous day close should be greater than next predicted day close any where
      # in the first quarter of the day atleast once
      if row.Close < Q1_prevdayclose:
        df.loc[df.Yfinance == i, 'DayBreak_check'] = 1
        #correct condition.
        break
  for index,row in S2.iterrows():
    # Midday conditions if Bullish, the close of the first quarter should be less than the
    # close anywhere in the second half of the day atleast once
    if check.MidDay.iloc[0] == 'Bullish':
      if row.Close > Q1_close:
        df.loc[df.Yfinance == i, 'MidDay_check'] = 1
        break
    elif check.MidDay.iloc[0] == 'Bearish':
      # Midday conditions if Bearish, the close of the first quarter should be greater than the
      # close anywhere in the second half of the day atleast once
      if row.Close < Q1_close:
        df.loc[df.Yfinance == i, 'MidDay_check'] = 1
        break
  print(dt[list(dt).index(chck) + 1])

clear_output(wait=False)
print('done')
# Note: the foward testing data displayed on record section of
# Infiflux is the performance on that particular day
# of the prediction made in the past.
# for example 2024-05-09 accuracy be xyz, this accuracy is for the prediction made by forecasts
# made by immediate previous working day in this case is 2024-05-08
# Another example, for monday, in case of Indian stock exchanges is forecast were made on friday.

done


In [ ]:
pd.DataFrame([[
    df.DayBreak_check.sum()/(len(df_li_tmp.keys())) * 100,
    df.MidDay_check.sum()/(len(df_li_tmp.keys())) * 100,
     (df.DayBreak_check.sum()/(len(df_li_tmp.keys())) + df.MidDay_check.sum()/(len(df_li_tmp.keys())))/2 * 100,
    len(df_li_tmp.keys())
    ]], columns = ['DayBreak_accuracy', 'MidDay_accuracy', 'Average of the two', 'no of forecasts'])


,DayBreak_accuracy,MidDay_accuracy,Average of the two,no of forecasts
0,85.714286,98.571429,92.142857,70
